In [37]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import glob
import json

import pandas as pd

In [39]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    json_file = filename.replace('backtest_result.txt','live_config.json')
    with open(json_file) as f:
        live_config = json.load(f)
        config_type = live_config['config_name']

    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())

        d['prod'] = False
        big_data = big_data.append(d, ignore_index=True)

files = glob.iglob('backtests/binance_production/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    json_file = filename.replace('backtest_result.txt','live_config.json')
    with open(json_file) as f:
        live_config = json.load(f)
        config_type = live_config['config_name']

    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance_production','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())

        d['prod'] = True
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_272032/266893688.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_272032/266893688.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_272032/266893688.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_272032/266893688.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_inde

,file,v,s,long,tgl_%,adgl_%,pamaxl,bnkrl_%,ebrl,maxsl,short,tgs_%,adgs_%,pamaxs,bnkrs_%,ebrs,maxss,prod
0,\DOTUSDT\plots\2022-04-03T085136\backtest_resu...,5.5,DOTUSDT,True,36.940,0.1290,0.725074,100.00,0.8642,11.102083,True,24.320,0.0670,0.474687,100.00,0.80150,35.380542,False
1,\DOTUSDT\plots\2022-04-03T090217\backtest_resu...,5.5,DOTUSDT,True,20.150,0.0700,0.771475,100.00,0.9064,16.916667,True,20.780,0.0562,0.472476,100.00,0.80160,35.381250,False
2,\ETHUSDT\plots\2022-04-02T210517\backtest_resu...,5.5,ETHUSDT,True,25.430,0.0954,0.613138,100.00,0.8365,37.025708,True,22.730,0.0481,0.251212,100.00,0.88510,16.301375,False
3,\ETHUSDT\plots\2022-04-02T220040\backtest_resu...,5.5,ETHUSDT,True,9.727,0.0392,0.558225,100.00,0.9095,30.801375,True,18.390,0.0490,0.272811,100.00,0.91560,17.956958,False
4,\FTMUSDT\plots\2022-04-03T084031\backtest_resu...,5.5,FTMUSDT,True,89.780,0.2730,1.193860,100.00,0.8551,14.206250,True,-54.870,0.2760,0.772471,7.45,0.09939,32.865292,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,\XMRUSDT\backtest_result.txt,5.5,XMRUSDT,True,4.557,0.0178,1.329670,100.00,0.8897,182.662500,True,16.830,0.0344,0.440727,100.00,0.84640,135.892917,True
94,\XRPUSDT\backtest_result.txt,5.5,XRPUSDT,True,-50.600,-1.6800,0.344107,5.99,0.8771,233.248750,True,3.811,-0.0122,0.623729,100.00,0.68000,251.355000,True
95,\ZENUSDT\backtest_result.txt,5.5,ZENUSDT,True,41.930,0.1300,0.968012,100.00,0.8677,14.339583,True,12.430,0.0286,0.420619,100.00,0.75650,32.212500,True
96,\ZILUSDT\backtest_result.txt,5.5,ZILUSDT,True,3.347,0.0125,1.171540,100.00,0.8991,35.809042,True,16.280,-0.0436,0.676679,100.00,0.63660,31.942375,True


Now filter the dataframe

In [40]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = ["ETH", "GALA","XRP", "WAVES","RUNE","FTM","IMX","DOT" ]
allowed_symbols = [s + "USDT" for s in allowed_symbols]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              #& (big_data['long'] == True)
              #& (big_data['adgl_%'] > 0.04)
              #& (big_data['short'] == True)
              #& (big_data['adgt_%'] > 0.1)
              #& (big_data['bnkrl_%'] > 95.0)
              #& (big_data['bnkrs_%'] > 95.0)
              #& (big_data['ebrs'] > .85)
              #& (big_data['ebrl'] > .85)
              #& (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,pamaxl,bnkrl_%,ebrl,maxsl,short,tgs_%,adgs_%,pamaxs,bnkrs_%,ebrs,maxss,prod,adgt_%
0,\DOTUSDT\plots\2022-04-03T085136\backtest_resu...,5.5,DOTUSDT,True,36.940,0.1290,0.725074,100.000,0.8642,11.102083,True,24.320,0.0670,0.474687,100.000,0.80150,35.380542,False,0.1960
1,\DOTUSDT\plots\2022-04-03T090217\backtest_resu...,5.5,DOTUSDT,True,20.150,0.0700,0.771475,100.000,0.9064,16.916667,True,20.780,0.0562,0.472476,100.000,0.80160,35.381250,False,0.1262
2,\ETHUSDT\plots\2022-04-02T210517\backtest_resu...,5.5,ETHUSDT,True,25.430,0.0954,0.613138,100.000,0.8365,37.025708,True,22.730,0.0481,0.251212,100.000,0.88510,16.301375,False,0.1435
3,\ETHUSDT\plots\2022-04-02T220040\backtest_resu...,5.5,ETHUSDT,True,9.727,0.0392,0.558225,100.000,0.9095,30.801375,True,18.390,0.0490,0.272811,100.000,0.91560,17.956958,False,0.0882
4,\FTMUSDT\plots\2022-04-03T084031\backtest_resu...,5.5,FTMUSDT,True,89.780,0.2730,1.193860,100.000,0.8551,14.206250,True,-54.870,0.2760,0.772471,7.450,0.09939,32.865292,False,0.5490
5,\FTMUSDT\plots\2022-04-03T090001\backtest_resu...,5.5,FTMUSDT,True,62.810,0.2030,1.267070,100.000,0.9255,17.521542,True,-93.210,-1.8800,0.818578,5.961,0.07093,206.972083,False,-1.6770
6,\GALAUSDT\plots\2022-04-02T215303\backtest_res...,5.5,GALAUSDT,True,44.370,0.1090,1.791730,100.000,0.7139,69.901250,True,-18.640,-0.0356,0.577101,61.080,0.52410,8.920833,False,0.0734
7,\GALAUSDT\plots\2022-04-02T220220\backtest_res...,5.5,GALAUSDT,True,14.030,0.0588,1.184470,100.000,0.9054,12.768750,True,-12.110,-0.0245,0.586079,100.000,0.67360,8.823625,False,0.0343
8,\IMXUSDT\plots\2022-04-03T084426\backtest_resu...,5.5,IMXUSDT,True,17.220,0.2790,0.497324,100.000,0.8193,8.309042,True,3.279,0.0913,0.468071,100.000,0.81910,15.425000,False,0.3703
9,\IMXUSDT\plots\2022-04-03T090112\backtest_resu...,5.5,IMXUSDT,True,13.200,0.2170,0.376271,100.000,0.9259,5.886125,True,2.031,0.0703,0.473151,100.000,0.81340,15.450000,False,0.2873


In [41]:
df.drop_duplicates(
    subset=['s', 'prod', 'tgl_%', 'tgs_%', 'adgl_%', 'adgs_%', 'adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long',
            'short'], keep='first',
    inplace=True)
dx = df[['file', 'prod', 's', 'adgt_%', 'adgl_%', 'adgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'maxsl', 'pamaxl',
         'maxss', 'pamaxs', ]]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_272032/1319857292.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(


,file,prod,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,maxss,pamaxs
0,\DOTUSDT\plots\2022-04-03T085136\backtest_resu...,False,DOTUSDT,0.1960,0.1290,0.0670,100.000,100.000,0.8642,0.80150,11.102083,0.725074,35.380542,0.474687
1,\DOTUSDT\plots\2022-04-03T090217\backtest_resu...,False,DOTUSDT,0.1262,0.0700,0.0562,100.000,100.000,0.9064,0.80160,16.916667,0.771475,35.381250,0.472476
2,\ETHUSDT\plots\2022-04-02T210517\backtest_resu...,False,ETHUSDT,0.1435,0.0954,0.0481,100.000,100.000,0.8365,0.88510,37.025708,0.613138,16.301375,0.251212
3,\ETHUSDT\plots\2022-04-02T220040\backtest_resu...,False,ETHUSDT,0.0882,0.0392,0.0490,100.000,100.000,0.9095,0.91560,30.801375,0.558225,17.956958,0.272811
4,\FTMUSDT\plots\2022-04-03T084031\backtest_resu...,False,FTMUSDT,0.5490,0.2730,0.2760,100.000,7.450,0.8551,0.09939,14.206250,1.193860,32.865292,0.772471
5,\FTMUSDT\plots\2022-04-03T090001\backtest_resu...,False,FTMUSDT,-1.6770,0.2030,-1.8800,100.000,5.961,0.9255,0.07093,17.521542,1.267070,206.972083,0.818578
6,\GALAUSDT\plots\2022-04-02T215303\backtest_res...,False,GALAUSDT,0.0734,0.1090,-0.0356,100.000,61.080,0.7139,0.52410,69.901250,1.791730,8.920833,0.577101
7,\GALAUSDT\plots\2022-04-02T220220\backtest_res...,False,GALAUSDT,0.0343,0.0588,-0.0245,100.000,100.000,0.9054,0.67360,12.768750,1.184470,8.823625,0.586079
8,\IMXUSDT\plots\2022-04-03T084426\backtest_resu...,False,IMXUSDT,0.3703,0.2790,0.0913,100.000,100.000,0.8193,0.81910,8.309042,0.497324,15.425000,0.468071
9,\IMXUSDT\plots\2022-04-03T090112\backtest_resu...,False,IMXUSDT,0.2873,0.2170,0.0703,100.000,100.000,0.9259,0.81340,5.886125,0.376271,15.450000,0.473151
